In [2]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install bert-tensorflow
!pip install tensorflow
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30508 sha256=1dab0207351f2b0027d487e868515a95f1f0af8998b25b063e915cf603713cb3
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19455 sha256=7e349f184e3cef025e1457524b76e2933237810bd6358a7ce3487472a8201459
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=135705537913de7ff83d44bce02f7a2fc6ff6bd6c25d0efd1b830d10d47462f5
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e908768

In [3]:
# from bert import tokenization
# from bert.tokenization.bert_tokenization import FullTokenizer
# import os
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [4]:
tokenizer.tokenize("I can't wait to visit Bulgaria again!")

['i', 'can', "'", 't', 'wait', 'to', 'visit', 'bulgaria', 'again', '!']

In [5]:
tokens = tokenizer.tokenize("I can't wait to visit Bulgaria again!")
tokenizer.convert_tokens_to_ids(tokens)

[1045, 2064, 1005, 1056, 3524, 2000, 3942, 8063, 2153, 999]

In [6]:
import pandas as pd

In [7]:
train = pd.read_csv("train_data_on.csv")
valid = pd.read_csv("val_data_on.csv")
test = pd.read_csv("test_data_on.csv")
train = train.append(valid).reset_index(drop=True)

<ipython-input-7-d82631b0785e>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(valid).reset_index(drop=True)


In [13]:
test

,text,intent
0,how can user profiling help businesses\n bette...,benefits_user_profilling
1,how can average time spent be used to identify...,average_time_spent
2,explain prompter,prompter_oconnect
3,how can i generate a ticket,ticket_creation_onpassive_ecosystem_chat
4,what is the maximum length of time that messag...,duration_trash_folder
...,...,...
561,want to access my photos and videos,my_media_onet
562,clarify cohost,co_host_oconnect
563,in what ways can a heat map help businesses un...,heat_map_definition
564,what does the total clicks metric represent in...,total_clicks_otrim


In [ ]:
train.drop("Unnamed: 0",axis=1,inplace=True)

In [14]:
train

,text,intent
0,what is the definition of \npage views in the ...,page_views
1,spam is a category in which certain emails are...,emails_goingto_spam
2,what is the purpose of showing unique clicks i...,unique_clicks
3,can you provide an example of how city click c...,city_clickcount
4,is there a way to tag a friend how,tag_untag_a_friend_onet
...,...,...
2637,define cohost,co_host_oconnect
2638,withdrawable credits,withdrawable_credits
2639,start the virtual bg meaning,process_to_select_virtual_bg_oconnect
2640,what are the stepsfor modifying a lengthy url,update_long_url


In [15]:
import numpy as np
from tqdm import tqdm

class IntentDetectionData:
    DATA_COLUMN = "text"
    LABEL_COLUMN = "intent"

    def __init__(
        self,
        train,
        test,
        tokenizer,
        classes,
        max_seq_len=192
    ):
        self.tokenizer = tokenizer
        self.max_seq_len = 0
        self.classes = classes
        data = self._prepare(train, test)
        if len(data) != 2:
            raise ValueError("The _prepare method should return exactly two values.")
        (self.train_x, self.train_y), (self.test_x, self.test_y) = data
        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        self.train_x, self.test_x = map(
            self._pad,
            [self.train_x, self.test_x]
        )

    def _prepare(self, train, test):
        train_x, train_y = [], []
        test_x, test_y = [], []
        unknown_labels = set()  # To store labels not found in classes

        for _, row in tqdm(train.iterrows()):
            text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
            if label not in self.classes:
                unknown_labels.add(label)
                continue  # Skip this data point
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            train_x.append(token_ids)
            train_y.append(self.classes.index(label))

        for _, row in tqdm(test.iterrows()):
            text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
            if label not in self.classes:
                unknown_labels.add(label)
                continue  # Skip this data point
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            test_x.append(token_ids)
            test_y.append(self.classes.index(label))

        if unknown_labels:
            print(f"Warning: Unknown labels found: {', '.join(unknown_labels)}")

        return (np.array(train_x), np.array(train_y)), (np.array(test_x), np.array(test_y))

    def _pad(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)


In [16]:
distilbert_model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [17]:
classes = train.intent.unique().tolist()
data = IntentDetectionData(
  train,
  test,
  tokenizer,
  classes,
  max_seq_len=128
)

2642it [00:00, 2892.22it/s]
566it [00:00, 2624.66it/s]

max seq_len 31



<ipython-input-15-7b9e0600f0db>:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (np.array(train_x), np.array(train_y)), (np.array(test_x), np.array(test_y))


In [25]:
data

In [20]:
!pip install tensorflow

In [24]:
import tensorflow as tf

# Sample text input
text = data

# Tokenize the text
input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, padding=True, return_tensors="tf")

# Perform inference using the DistilBERT model
outputs = distilbert_model(input_ids)

# Get the predicted logits
logits = outputs.logits

# If you want probabilities, you can apply a softmax
probabilities = tf.nn.softmax(logits, axis=1)

# Convert the probabilities to numpy for easy handling
probabilities = probabilities.numpy()

# The probabilities will give you the predicted class probabilities for your input text
print(probabilities)


ValueError: ignored

In [ ]:
model = distilbert_model(data.max_seq_len)

ValueError: ignored

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 31)]              0         
                                                                 
 tf_bert_for_sequence_classi  TFSequenceClassifierOutp  109483778
 fication (TFBertForSequence  ut(loss=None, logits=(No           
 Classification)             ne, 2),                             
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
 dense (Dense)               (None, 347)               1041      
                                                                 
Total params: 109,484,819
Trainable params: 1,041
Non-trainable params: 109,483,778
_________________________________________________________________


In [ ]:
import keras
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import datetime
log_dir = "log/intent_detection/" +\
datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
model.fit(
  x=data.train_x,
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=5,
  callbacks=[tensorboard_callback]
)

Epoch 1/5
181/181 [==============================] - 31s 78ms/step - loss: 5.8524 - accuracy: 0.0031 - val_loss: 5.8563 - val_accuracy: 0.0000e+00
Epoch 2/5
181/181 [==============================] - 11s 64ms/step - loss: 5.8463 - accuracy: 0.0048 - val_loss: 5.8601 - val_accuracy: 0.0000e+00
Epoch 3/5
181/181 [==============================] - 11s 59ms/step - loss: 5.8437 - accuracy: 0.0052 - val_loss: 5.8642 - val_accuracy: 0.0000e+00
Epoch 4/5
181/181 [==============================] - 11s 58ms/step - loss: 5.8413 - accuracy: 0.0052 - val_loss: 5.8678 - val_accuracy: 0.0000e+00
Epoch 5/5
181/181 [==============================] - 10s 54ms/step - loss: 5.8392 - accuracy: 0.0052 - val_loss: 5.8711 - val_accuracy: 0.0000e+00


In [ ]:
_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)
print("train acc", train_acc)
print("test acc", test_acc)

18/18 [==============================] - 2s 84ms/step - loss: 5.8498 - acc: 0.0018
train acc 0.0012464942410588264
test acc 0.0017699114978313446


In [23]:
import tensorflow as tf

# Sample text input
text = "hi how are you"

# Tokenize the text
input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, padding=True, return_tensors="tf")

# Perform inference using the DistilBERT model
outputs = distilbert_model(input_ids)

# Get the predicted logits
logits = outputs.logits

# If you want probabilities, you can apply a softmax
probabilities = tf.nn.softmax(logits, axis=1)

# Convert the probabilities to numpy for easy handling
probabilities = probabilities.numpy()

# Define your class labels (replace with your actual labels)
class_labels = ["greet", "good_bye", "sorry"]

# Find the index of the class with the highest probability
predicted_class_index = tf.argmax(probabilities, axis=1)[0].numpy()

# Get the predicted label
predicted_label = class_labels[predicted_class_index]

# The probabilities will give you the predicted class probabilities for your input text
print("Predicted Class Probabilities:", probabilities)
print("Predicted Label:", predicted_label)


Predicted Class Probabilities: [[0.4747309  0.52526915]]
Predicted Label: good_bye
